### Hate and Toxic Comment Detection 

**DataLab USAL - Project 2019/2020** 

### 01. Corpus Exploration

Este primer Notebook servirá como primera toma de contacto. Cargaremos los tweets y daremos el formato adecuado a la información de la que disponemos. Además, realizaremos un análisis exploratorio básico sobre las variables con algunos gráficos descriptivos.

---

#### Librerías

In [3]:
# standard
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# misc
import gzip
import logging
import codecs
global lcode
import warnings
warnings.filterwarnings("ignore")

# nlp
import re
from collections import Counter
from nltk import word_tokenize
import nltk
import spacy
import itertools
import collections
from nltk.corpus import stopwords
import networkx
from gensim import models

from gensim.models.keyedvectors import KeyedVectors
from gensim.models import Word2Vec
from scipy.spatial import distance
import yaml
import re
from sklearn.cluster import KMeans
from gensim.test.utils import common_texts , get_tmpfile

# pandas parameters
pd.set_option('display.max_columns', None)

# seaborn parameters
sns.set(font_scale=1.5)
sns.set_style("whitegrid")

%matplotlib inline

### Lectura de tweets

In [4]:
col_names = ["col1", "date", "user", "tweet", "via", "id", "followers", "following",
             "statuses", "loc", "link", "col12", "col13", "user_bio", "col15", "col16",
             "col17", "tweet_type", "col19", "politician_mention", "col21", "politician_rt", "col23", "col24",
             "hashtag", "col26","count_creation", "col28", "col29", "col30"]

In [6]:
files = os.listdir('../DATA/')

In [8]:
tweets = pd.read_csv('../DATA/streaming_odio_politicas_0.txt', sep= "\t",
                  names=col_names, encoding='utf-8')

In [10]:
tweets_dataframe = pd.DataFrame()

for file_name in files:
    tweets = pd.read_csv("../DATA/" + file_name, sep="\t", names=col_names, encoding='utf-8')
    tweets_dataframe = pd.concat([tweets_dataframe, tweets])

In [12]:
tweets_dataframe.head()

,col1,date,user,tweet,via,id,followers,following,statuses,loc,link,col12,col13,user_bio,col15,col16,col17,tweet_type,col19,politician_mention,col21,politician_rt,col23,col24,hashtag,col26,count_creation,col28,col29,col30
0,version https://git-lfs.github.com/spec/v1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,oid sha256:b772244634a78cd13007a24668c264f9eb5...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,size 413610152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1163184609351340033,2019-08-18 20:23:16,@Gas19861,RT @Santi_ABASCAL: Apalean a cuatro turistas a...,via=Twitter for Android,id=358194398,followers=349,following=1649,statuses=4604,"loc=Mérida, España",https://www.elmundo.es/cataluna/2019/08/17/5d5...,None,Gas1986,#ElAireLibreNosHaceLibres #E&M=mM&mL. Maquinis...,None,None,None,RT,None,None,1162974082566172672,Santi_ABASCAL,None,None,None,es,2011-08-19 14:48:06,False,https://pbs.twimg.com/profile_images/103315113...,https://twitter.com/Gas19861/status/1163184609...
1,1163184609607213057,2019-08-18 20:23:16,@Pablopc1985,RT @Santi_ABASCAL: Apalean a cuatro turistas a...,via=Twitter for iPhone,id=954768010719432704,followers=41,following=125,statuses=1027,loc=None,https://www.elmundo.es/cataluna/2019/08/17/5d5...,None,Pablo,None,None,None,None,RT,None,None,1162974082566172672,Santi_ABASCAL,None,None,None,es,2018-01-20 17:30:07,False,https://pbs.twimg.com/profile_images/954848790...,https://twitter.com/Pablopc1985/status/1163184...


### Eliminar variables inservibles y renombrar

In [13]:
tweets.drop(["col1", "id", "statuses", "col13", "col15", "col16", "col19", 
             "col21", "col23", "col26", "col29", "col30"], axis=1, inplace=True)

In [14]:
tweets.rename(columns={"col2": "date",
                       "col3": "user",
                       "col11": "link",
                       "col14": "user_bio",
                       "col18": "tweet_type",
                       "col20": "politician_mention",
                       "col22": "politician_rt",
                       "col25": "hashtag",
                       "col27": "count_creation"}, inplace=True)

In [15]:
tweets.head(5)

,date,user,tweet,via,followers,following,loc,link,col12,user_bio,col17,tweet_type,politician_mention,politician_rt,col24,hashtag,count_creation,col28
0,2019-09-08 11:06:52,@delsol1469,@Irene_Montero_ https://t.co/g9PBGfaohx,via=Twitter for Android,followers=15,following=76,loc=None,None,None,None,Una feminazi tan fea que levantaría a Chanquet...,quote,None,None,delsol1469,None,2012-04-21 18:42:14,False
1,2019-09-08 11:06:53,@Almuden77069881,"@sanchezcastejon Por favor, si tienes votos y ...",via=Twitter for Android,followers=624,following=1435,loc=navarra,None,None,Amante de los libros y de la verdad Indignada ...,None,reply,sanchezcastejon,None,None,None,2012-03-06 14:44:59,False
2,2019-09-08 11:06:57,@PostTruth2020,"@cayetanaAT Por mucho menos, Irene Montero se ...",via=Twitter for iPhone,followers=1280,following=1070,loc=España,None,None,no poseo la verdad; la verdad me posee!,None,reply,cayetanaAT,None,None,None,2017-10-08 19:52:02,False
3,2019-09-08 11:07:13,@delsol1469,@sanchezcastejon https://t.co/g9PBGfaohx,via=Twitter for Android,followers=15,following=76,loc=None,None,None,None,Una feminazi tan fea que levantaría a Chanquet...,quote,None,None,delsol1469,None,2012-04-21 18:42:14,False
4,2019-09-08 11:07:14,@j_estarellas,RT @cayetanaAT: Reclamamos una Alta Inspección...,via=Twitter for iPad,followers=15,following=256,"loc=Madrid, Comunidad de Madrid",https://www.abc.es/sociedad/abci-ofensiva-pp-p...,None,Director de SHD ‘Proyecto Español para Combati...,None,RT,None,cayetanaAT,None,None,2019-08-08 18:27:08,False


### Limpieza de variables informativas

In [16]:
tweets["via"] = tweets["via"].str.replace("via=", "")
tweets["followers"] = tweets["followers"].str.replace("followers=", "")
tweets["following"] = tweets["following"].str.replace("following=", "")
tweets["loc"] = tweets["loc"].str.replace("loc=", "")

### Formato de variables

In [17]:
tweets["date"] = pd.to_datetime(tweets["date"], format="%Y-%m-%d %H:%M:%S", errors="coerce")
tweets["date"].dtype

dtype('<M8[ns]')

In [18]:
tweets["followers"] = pd.to_numeric(tweets["followers"], errors="coerce")
tweets["followers"].dtype

dtype('int64')

In [19]:
tweets["following"] = pd.to_numeric(tweets["following"], errors="coerce")
tweets["following"].dtype

dtype('int64')

### Limpieza de tweets

In [20]:
tweets["tweet"] = tweets["tweet"].astype(str)

In [21]:
politicians = ["@InesArrimadas", "@Irene_Montero_", "@carmencalvo_", "@cayetanaAT", 
               "@Albert_Rivera", "@Pablo_Iglesias_", "@sanchezcastejon", "@pablocasado_",
               "@Santi_ABASCAL", "@monasterioR"]

In [22]:
def tweet_cleaning(df):
    for polit in politicians:
        df = df.apply(lambda elem: " ".join(re.sub(polit, "", elem).split()))
        
    return df

In [23]:
def tweet_cleaning2(df):
    df = df.str.lower()
    df = df.apply(lambda elem: " ".join(re.sub("á", "a", elem).split()))
    df = df.apply(lambda elem: " ".join(re.sub("é", "e", elem).split()))
    df = df.apply(lambda elem: " ".join(re.sub("í", "i", elem).split()))
    df = df.apply(lambda elem: " ".join(re.sub("ó", "o", elem).split()))
    df = df.apply(lambda elem: " ".join(re.sub("ú", "u", elem).split()))
    df = df.apply(lambda elem: " ".join(re.sub("ñ", "n", elem).split()))
    df = df.apply(lambda elem: " ".join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|http.+?|^rt", " ", elem).split()))
    return df

In [24]:
def clean_text(text):
    regex_dict = { r'á' : 'a', 'é' : 'e', 'í' : 'i', 'ó' : 'o', 'ú' : 'u' , '[^\w#]' : ' ', '[ 0-9 ]' : ''}
    # Create a regular expression  from the dictionary keys
    if isinstance(text, str):
        regex = re.compile(r'(%s)' % "|".join(regex_dict.keys()))
        lst = regex.sub(lambda mo: regex_dict[[ k for k in regex_dict if re.search(k, mo.string[mo.start():mo.end()])][0]], text).lower()
    else:
        lst = ' '
    return lst

In [26]:
# tweets_dataframe.loc[:,'tweet'] = tweets_dataframe.loc[:,'tweet'].apply(tweet_cleaning2)

KeyboardInterrupt: 

In [27]:
tweets['tweet'] = tweet_cleaning2(tweets['tweet'])

In [227]:
#tweets['tweet'] = clean_text(tweets['tweet'])

In [28]:
tweets["tweet"]

0                                                   montero
1         por favor si tienes votos y escanos x q no est...
2         por mucho menos irene montero se ha sacado 3 0...
3                                                          
4         reclamamos una alta inspeccion con la fuerza s...
5         rivera muchas felicidades a todos los asturian...
6         sindicatos y padres de alumnos exigen derogar ...
7         ante el chantaje y el bloqueo de sanchez propo...
8         rivera que noticia tan triste el fallecimiento...
9                           no sabes ni lo que lees maquina
10        con su voz genuina como interprete y composito...
11        le digo a unidas podemos que asumamos nuestros...
12                                              uno del pnv
13        dedican su vida a ayudar a quienes lo necesita...
14        montero honestidad sabes que es eso bueno no t...
15        iglesias 11 de septiembre 15h hall hospital de...
16        desde el dia que fallecio hay 

#### Información del dataset

In [29]:
print("Número de tweets recogidos: " + str(tweets.shape[0]))
print("Variables informativas/tweet: " + str(tweets.shape[1]))

Número de tweets recogidos: 121954
Variables informativas/tweet: 18


## Stop Words

In [30]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words_set = set(stopwords.words("spanish"))

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
stop_words = list(stop_words_set)
stop_words

['mía',
 'mías',
 'nos',
 'nuestros',
 'habidos',
 'ese',
 'somos',
 'es',
 'sería',
 'las',
 'serías',
 'fuiste',
 'fuimos',
 'estáis',
 'y',
 'habrían',
 'tuviéramos',
 'un',
 'han',
 'sea',
 'estuvieses',
 'hubo',
 'unos',
 'que',
 'hubieran',
 'hubiéramos',
 'hubieseis',
 'fue',
 'hube',
 'esos',
 'estaría',
 'hemos',
 'son',
 'nuestra',
 'estén',
 'le',
 'tanto',
 'vuestras',
 'tuviste',
 'me',
 'contra',
 'esté',
 'habrá',
 'les',
 'mío',
 'los',
 'quien',
 'tendrá',
 'ti',
 'hayáis',
 'suyo',
 'con',
 'quienes',
 'hubieras',
 'fuera',
 'hubiésemos',
 'tuviésemos',
 'fuésemos',
 'habréis',
 'también',
 'estoy',
 'su',
 'ellos',
 'teniendo',
 'tengan',
 'había',
 'vosotras',
 'serían',
 'sois',
 'está',
 'estábamos',
 'estarían',
 'estuviéramos',
 'tengas',
 'el',
 'tienen',
 'algunas',
 'por',
 'suyos',
 'erais',
 'vuestro',
 'tiene',
 'esta',
 'nada',
 'ella',
 'todo',
 'tenemos',
 'tuvimos',
 'te',
 'estés',
 'sobre',
 'estabas',
 'fueras',
 'todos',
 'has',
 'tendrían',
 'no',

In [32]:
def stop_cleaning(stop_word_list):
    words_cleaned = [w.replace('á', 'a') for w in stop_word_list]
    words_cleaned = [w.replace('é', 'e') for w in words_cleaned]
    words_cleaned = [w.replace('í', 'i') for w in words_cleaned]
    words_cleaned = [w.replace('ó', 'o') for w in words_cleaned]
    words_cleaned = [w.replace('ú', 'u') for w in words_cleaned]
    return words_cleaned

In [33]:
stopss = stop_cleaning(stop_words)
stopss

['mia',
 'mias',
 'nos',
 'nuestros',
 'habidos',
 'ese',
 'somos',
 'es',
 'seria',
 'las',
 'serias',
 'fuiste',
 'fuimos',
 'estais',
 'y',
 'habrian',
 'tuvieramos',
 'un',
 'han',
 'sea',
 'estuvieses',
 'hubo',
 'unos',
 'que',
 'hubieran',
 'hubieramos',
 'hubieseis',
 'fue',
 'hube',
 'esos',
 'estaria',
 'hemos',
 'son',
 'nuestra',
 'esten',
 'le',
 'tanto',
 'vuestras',
 'tuviste',
 'me',
 'contra',
 'este',
 'habra',
 'les',
 'mio',
 'los',
 'quien',
 'tendra',
 'ti',
 'hayais',
 'suyo',
 'con',
 'quienes',
 'hubieras',
 'fuera',
 'hubiesemos',
 'tuviesemos',
 'fuesemos',
 'habreis',
 'tambien',
 'estoy',
 'su',
 'ellos',
 'teniendo',
 'tengan',
 'habia',
 'vosotras',
 'serian',
 'sois',
 'esta',
 'estabamos',
 'estarian',
 'estuvieramos',
 'tengas',
 'el',
 'tienen',
 'algunas',
 'por',
 'suyos',
 'erais',
 'vuestro',
 'tiene',
 'esta',
 'nada',
 'ella',
 'todo',
 'tenemos',
 'tuvimos',
 'te',
 'estes',
 'sobre',
 'estabas',
 'fueras',
 'todos',
 'has',
 'tendrian',
 'no',

In [34]:
custom_stop_words=['q', 'k', 'tiene','vuestra','habiendo','han','tendremos','tenemos','hubisteis','hayas',
        'habian','seas','muy','se','tienes','habidos','mia','tuviste','tengas','contra','tuvieseis',
        'habiais','he','estariamos','estuvieron','para','fui','has','al','era','tenidos','nada',
        'estados','tuvieran','teneis','sentidas','fueran','estare','en','sois','que','estuvieras',
        'tendras','tuvieses','estado','erais','sus','si','seais','estuvieramos','hubimos','tuyas',
        'esteis','quienes','ella','todos','sea','fueses','otra','tenido','mios','del','sean','tenia',
        'mas','tambien','fueseis','ni','vuestro','habiamos','fueras','tu','que','sentida','antes',
        'y','hube','tendrian','tu','estarian','este','tuvierais','os','tendre','no','fueramos','vosotras',
        'ti','tendrias','seran','habriais','esta','estos','lo','estuviesemos','hubieses','eres',
        'nuestra','hayamos','estaras','la','estamos','habria','somos','tendran','yo','habras','habra',
        'seria','hay','poco','tendriamos','por','el','pero','habrias','estuviste','sobre','otro',
        'estaran','habeis','esos','tuvieron','hayais','cuando','estadas','estuvo','tenias','estemos',
        'sintiendo','teniamos','habias','tuvo','hubiese','ha','vuestras','son','esto','nosotros',
        'estaba','como','habre','habreis','mucho','habrian','vosotros','eras','fue','sentid','sera',
        'mi','estas','ante','es','tuvieramos','mias','me','hubieras','tenian','esta','estuvieses',
        'hubiesemos','tuve','fuese','estuvierais','los','hubieron','desde','fuiste','cual','estuve',
        'tus','mis','ya','nuestros','tuviese','donde','su','eran','ese','serias','habidas','durante',
        'habran','tengamos','tuvimos','le','estaban','seamos','estarias','un','algo','tengan','sereis',
        'estabamos','tuviesemos','estara','hubieseis','serian','seriais','tenida','estabais','estuvimos',
        'estuviesen','mio','fueron','siente','ellos','e','suya','estuviese','estuvieran','nosotras',
        'estuvieseis','o','tienen','las','habriamos','estuviera','tengo','estando','muchos','tenidas',
        'tendra','estaremos','el','hubo','estaria','sere','a','vuestros','hubierais','suyo','tuvisteis',
        'otros','tuvieras','eramos','estes','habido','estoy','suyas','estabas','fuerais','algunas',
        'esten','soy','otras','tanto','estariais','estareis','hasta','habremos','esa','hayan','mi',
        'eso','quien','tened','estan','tuya','hubieran','entre','seriamos','unos','hubiera','porque',
        'algunos','estas','fuera','estar','habida','fuesen','tendreis','con','tengais','suyos',
        'este','estad','fuisteis','sentido','todo','hubieramos','estuvisteis','sin','seremos','tuyos',
        'de','uno','les','nuestro','sentidos','teniais','hubiesen','hubiste','tuviesen','esas','tendria',
        'seras','teniendo','estais','te','tuyo','fuimos','haya','fuesemos','habia','nuestras','tenga',
        'una','tendriais','tuviera','hemos','nos','ellas','estada']

### Recuento palabras más habituales

Función para tokenizar, eliminar stop words y hacer recuento de palabras más  habituales

In [35]:
def remove_stopwords(words):
    new_words = []
    for word in words:
        if word not in custom_stop_words:
            new_words.append(word)
    return new_words

In [36]:
tweets_no_stop = remove_stopwords(tweets["tweet"])

In [37]:
tweets_no_stop

['montero',
 'por favor si tienes votos y escanos x q no estas gobernando sencillamente no tienes nada y en noviembre te vas a quedar sin nada x prepotente y chuleta',
 'por mucho menos irene montero se ha sacado 3 000 euritmia mira a ver',
 '',
 'reclamamos una alta inspeccion con la fuerza sancionadora que el psoe le quito capaz de combatir las mentiras en los libros de texto y garantizar las ensenanzas comunes y el aprendizaje en espanol y nunca mas un informe en el cajon',
 'rivera muchas felicidades a todos los asturianos en el dia de su tierrina uno de los paraisos naturales de espana sus impresionantes paisajes su riqueza cultural gastronomica y artistica y el encanto de sus pueblos y sus gentes son un orgullo para todos feliz diadeasturias',
 'sindicatos y padres de alumnos exigen derogar la lomce y acabar con recortes a traves de',
 'ante el chantaje y el bloqueo de sanchez propongo a los barones socialistas y de cs que presiden cc aa que unan sus votos al pp para convocar el 

### Simple tokenization

In [38]:
# Requires `stop_words` and `tweets['clean_tweet']`

def simple_tokenization():
    tweets['tokens'] = tweets["tweet"].apply(lambda text: [word for word in text.split() if word not in custom_stop_words])
    return tweets['tokens']
    
    
tweets['tokens']=simple_tokenization()

In [39]:
tweets['tokens'].head(5)

0                                            [montero]
1    [favor, votos, escanos, x, gobernando, sencill...
2    [menos, irene, montero, sacado, 3, 000, euritm...
3                                                   []
4    [reclamamos, alta, inspeccion, fuerza, sancion...
Name: tokens, dtype: object

Tweets towenizados y sin stop_words

In [240]:
vector_size = 100
window_size = 4
min_count = 2
workers_num = 10

In [241]:
model_es = models.Word2Vec(size = vector_size, window = window_size, min_count = min_count, workers = workers_num)
model_es.build_vocab(tweets['tokens'])  # prepare the model vocabulary
model_es.train(tweets['tokens'], total_examples=model_es.corpus_count, epochs=500)  # train word vectors

(923226061, 980227500)

In [242]:
def doc2vec(text):
    """text and language and returns the mean of all word2vec of the text"""
    words_list = text
    model = model_es
    words_list = [word for word in words_list if word in model.wv.vocab]

    if len(words_list) > 0:
        return np.mean(model(model[words_list]), axis=0)
    else:
        return np.empty(vector_size)


In [243]:
# Computing doc2vecs... 

doc2vec_es = []

for index, row in tweets.iterrows():
    doc2vec_es.append(doc2vec(row[tweets['tokens']]))

TypeError: unhashable type: 'list'

In [ ]:
# Computing doc2vecs... 

doc2vec_proposals_es = []

word2vec_model_ca = Word2Vec.load('/Users/cxb0266/Desktop/data' + 'word2vec' + '.bin')

for index, row in proposals.iterrows():
    doc2vec_proposals_es.append(doc2vec(row['text_es'], 'es'))

In [ ]:
def grafico(count):
    clean_tweets_nsw = pd.DataFrame(count, columns=['words', 'count'])

    fig, ax = plt.subplots(figsize=(8, 8))

    # Plot horizontal bar graph
    clean_tweets_nsw.sort_values(by='count').plot.barh(x='words', y='count',
                                                       ax=ax, color="purple")

    ax.set_title("Common words found (without stop words)")

    return plt.show()

In [ ]:
grafico(count)

### Seleccionamos los retweets y vemos los # mas habituales

In [ ]:
tweets_RT = tweets.loc[tweets["tweet_type"] == "RT"]

In [ ]:
counts = collections.Counter(tweets_RT["hashtag"])
count = counts.most_common(10)
count

In [ ]:
tweets['date'] = pd.to_datetime(tweets['date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
tweets.head(5)

In [ ]:
tweets['date'] = pd.to_datetime(tweets['date']).dt.floor('d')
tweets.head(5)

In [ ]:
tweets_by_date = tweets['date'].value_counts().rename_axis('date').reset_index(name='count')
tweets_by_date = tweets_by_date.sort_values(by=['date'], )
tweets_by_date

In [ ]:
fig = plt.figure()
plt.plot(tweets_by_date['date'],tweets_by_date['count'])
plt.xlabel('DATES')
plt.ylabel('TWEETS COUNT')
plt.xticks(rotation=45)

In [ ]:
%%HTML
<img style="float: right;" src="datalab.png" width="400px"/>